In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',None)

In [2]:
ipl_df = pd.read_csv(r"C:\Users\ANAIKUTTY\Documents\Cricket analysis\data files\ipl_2008_2024_ball_by_ball_data.csv")

In [3]:
df = ipl_df.copy()

In [4]:
df = df[df['innings'].isin([1,2])]

In [5]:
df['tot_runs'] = df['runs_off_bat'] + df['extras']
df['isout'] = np.where(df['player_dismissed'].isna(),0,1)

In [6]:
t1 = 'Gujarat Titans'
t2 = 'Chennai Super Kings'

In [8]:
outcomes = [0,1,2,3,4,6,'w']

In [7]:
df[(df['batting_team'] == t1)&(df['bowling_team']==t2)&(df['innings']==1)]['match_id'].unique()

array([1370353, 1426297])

In [9]:
t1_outcome_count = df[df['batting_team']==t1]['tot_runs'].value_counts() #GT Batting
t2_outcome_count = df[df['batting_team']==t2]['tot_runs'].value_counts() #CSK Batting

In [10]:
t1_outs = df[df['batting_team']==t1]['isout'].sum()
t2_outs = df[df['batting_team']==t2]['isout'].sum()

In [11]:
t1_outcomes = []
t2_outcomes = []

for i in outcomes:
    if i != 'w':
        t1_outcomes.append(t1_outcome_count[i])
        t2_outcomes.append(t2_outcome_count[i])
    else:
        t1_outcomes.append(t1_outs)
        t2_outcomes.append(t2_outs)

In [12]:
t1_prob_outcomes = [i/sum(t1_outcomes) for i in t1_outcomes]
t2_prob_outcomes = [i/sum(t2_outcomes) for i in t2_outcomes]

In [13]:
t1_outcomes

[np.int64(1662),
 np.int64(2447),
 np.int64(387),
 np.int64(14),
 np.int64(698),
 np.int64(270),
 np.int64(247)]

In [14]:
t2_outcomes

[np.int64(9676),
 np.int64(12105),
 np.int64(1955),
 np.int64(111),
 np.int64(3240),
 np.int64(1499),
 np.int64(1243)]

In [15]:
t1_prob_outcomes , t2_prob_outcomes

([np.float64(0.2903056768558952),
  np.float64(0.4274235807860262),
  np.float64(0.06759825327510917),
  np.float64(0.002445414847161572),
  np.float64(0.12192139737991266),
  np.float64(0.04716157205240175),
  np.float64(0.04314410480349345)],
 [np.float64(0.3243823125146669),
  np.float64(0.40581313486875187),
  np.float64(0.06554024606926145),
  np.float64(0.0037212109021422107),
  np.float64(0.10861912903550236),
  np.float64(0.05025310939019075),
  np.float64(0.041670857219484395)])

In [16]:
def get_pbvalues(teamName):
  
    if teamName == 'GT':
        p_0 = 0.28941
        p_1 = 0.28941 + 0.43028
        p_2 = 0.28941 + 0.43028 + 0.06636
        p_3 = 0.28941 + 0.43028 + 0.06636 + 0.0
        p_4 = 0.28941 + 0.43028 + 0.06636 + 0.0 + 0.12314
        p_6 = 0.28941 + 0.43028 + 0.06636 + 0.0 + 0.12314 + 0.04839
        p_w = 1
    
    elif teamName == 'CSK':
        p_0 = 0.32710
        p_1 = 0.32710 + 0.40425
        p_2 = 0.32710 + 0.40425 + 0.06559
        p_3 = 0.32710 + 0.40425 + 0.06559 + 0.0
        p_4 = 0.32710 + 0.40425 + 0.06559 + 0.0 + 0.10803
        p_6 = 0.32710 + 0.40425 + 0.06559 + 0.0 + 0.10803 + 0.04964
        p_w = 1

    return p_0, p_1, p_2, p_3, p_4, p_6, p_w

In [17]:
def predict_runs(target, current_score, current_wickets, current_overs):
  
    # pb values of both teams
    i1p_0, i1p_1, i1p_2, i1p_3, i1p_4, i1p_6, i1p_w = get_pbvalues('GT')
    i2p_0, i2p_1, i2p_2, i2p_3, i2p_4, i2p_6, i2p_w = get_pbvalues('CSK')

    pred_runs = current_score
    pred_wks = current_wickets
    leftover_balls = 120 - current_overs*6

    for i in range(leftover_balls):
        r_value = np.random.random()

        if r_value <= i2p_0:
            pred_runs += 0
        elif r_value <= i2p_1:
            pred_runs += 1
        elif r_value <= i2p_2:
            pred_runs += 2
        elif r_value <= i2p_3:
            pred_runs += 3
        elif r_value <= i2p_4:
            pred_runs += 4
        elif r_value <= i2p_6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            if pred_wks == 10:
                break
        if pred_runs > target:
            break
        # print('pred_runs: ', pred_runs)
        # print('pred_wks: ', pred_wks)
    
    return pred_runs

In [18]:
predict_runs(171, 0, 0, 0)

172

In [19]:
def get_win(pred_runs, target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

In [20]:
# runs, wickets, overs, 
# win - 1st ing score

# GT - 214/4 
#reduced to 17 overs 171 to chase from 15 overs (DLS method)

target = 171

current_score = 133
current_wickets = 3
current_overs = 12

iter_count = 100

runs_ls = []
results_ls = []

win_count = 0
tie_count = 0
lose_count = 0

for i in range(iter_count):
    pred_runs = predict_runs(target, current_score, current_wickets, current_overs)
    runs_ls.append(pred_runs)
    result_pred = get_win(pred_runs, target)
    results_ls.append(result_pred)

    if result_pred == 'win':
        win_count += 1
    elif result_pred == 'tie':
        tie_count += 1
    else:
        lose_count +=1

In [21]:
win_count, tie_count, lose_count

(98, 0, 2)

In [22]:
def find_runs(current_score, target, current_wickets, at_overs):
    runs_ls = []
    results_ls = []
    req_runs = []
    win_ls = []

    for i in range(current_score, target + 1):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
            pred_runs = predict_runs(target, i, current_wickets, at_overs)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

            win_ls.append(win_count)
            req_runs.append(i)
            # print('runs: ', i, ' win%: ', win_count)

    required_runs = current_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            # print('Runs to be: ', req_runs[i])
            break

    return required_runs

In [23]:
find_runs(133, 171, 3, 12)

133

In [24]:
def find_wickets(current_score, target, current_wickets, at_overs):

#     find_runs(current_score, target, current_wickets, at_overs)
    req_runs = find_runs(current_score, target, current_wickets, at_overs)

    runs_ls = []
    results_ls = []

    req_wks = []
    win_ls = []

    for i in range(current_wickets, 10):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
#             pred_runs = predict_runs(target, req_runs, i, at_overs)
            pred_runs = predict_runs(target, current_score, i, at_overs)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

        win_ls.append(win_count)
        req_wks.append(i)
     #print('wickets: ', i, ' win%: ', win_count)

    req_wicket_value = current_wickets
    
    for i in range(len(req_wks)):
        if (win_ls[i] < 45)  :
            req_wicket_value = req_wks[i]
            break

    return req_wicket_value

In [26]:
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(current_wks, at_overs, target_score):
    plt.figure(figsize = (16, 6))
    # x = np.linspace(-10, 10, num=1000)
    x = np.array(list(range(21)))
    req_value = find_runs(133, target_score, current_wks, at_overs)
    req_wk_value = find_wickets(133, target_score, current_wks, at_overs)

    if at_overs == 12:
        req_value = 133
        req_wk_value = 3

    
    # print(req_value)
    y = np.array([req_value for i in range(21)])

    # plt.plot(x, current_overs * x + target_score)
    # plt.plot(x, y)
    plt.scatter(at_overs, req_value, s = 1200, color = 'red')
    plt.axhline(target_score, ls = '--', color = 'blue')
    plt.text( 1, target_score + 10, 'Target Score :' + str(target_score) , color = 'darkblue', fontsize = 13)
    plt.text( at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color = 'white', fontsize = 12,  horizontalalignment='center', verticalalignment='center')
    plt.text(at_overs, req_value - 30, 'CSK has to be at ' + str(req_value) + '/' +  str(req_wk_value) + ' after ' + str(at_overs) + ' ov', horizontalalignment='center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be?', fontsize = 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

# x=widgets.IntSlider(min=-10, max=30, step=1, value=10)

# find_wickets(current_score, target, current_wickets, at_overs)
# find_wickets(133, 171, 3, 12)